In [1]:
import numpy as np 
import pandas as pd 
import os

import matplotlib 
from glob import glob 
import open3d as o3d   
import math
matplotlib.use('TkAgg')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sys 
sys.path.append("../src/")
from utils.load_tof_images import create_from_zip_absolute  as load_assignment_data
from depth_model import inference as infer

id = "6295be80-2857-11ed-8783-4b26e63c0e02"  # Change the id to get the other values 
path = "../data/360_scan/"+id
rgb_files = glob(path+"/rgb/*")
print("Total RGB images ",len(rgb_files))

Fusing layers... 


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Total RGB images  26


In [3]:
# Depth camera parameters:
FY_DEPTH = 0.7811297
FX_DEPTH = 1.5166936
CY_DEPTH = 0.50329405
CX_DEPTH = 0.5187362


def get_image_data(rgb_fpath):
    depth_fpath = rgb_fpath.replace('rgb','depth')
    calib_fpath = os.path.dirname(rgb_fpath).replace('rgb','calibration/0')
    data = load_assignment_data(rgb_fpath=rgb_fpath,depthmap_fpath=depth_fpath,calibration_fpath=calib_fpath)
    return data[8],data[3],data[4]


def create_point_cloud(predicted_image):
    # get depth resolution:
    height, width = predicted_image.shape
    length = height * width

    # compute indices:
    jj = np.tile(range(width), height)
    ii = np.repeat(range(height), width)

    # reshape depth image
    z = predicted_image.reshape(length)
    
    # compute pcd:
    pcd = np.dstack([(ii - CX_DEPTH) * z / FX_DEPTH,
                    (jj - CY_DEPTH) * z / FY_DEPTH,
                    z]).reshape((length, 3))
    # print("Point Cloud shape ",pcd.shape)
    return pcd

def get_approx_height(pcd_points):
    x_max = max(pcd_points.points,key=lambda x: x[0])
    y_max = max(pcd_points.points,key=lambda x: x[1])
    z_max = max(pcd_points.points,key=lambda x: x[2])

    x_min = min(pcd_points.points,key=lambda x: x[0])
    y_min = min(pcd_points.points,key=lambda x: x[1])
    z_min = min(pcd_points.points,key=lambda x: x[2])
    
    height = math.sqrt(z_max[2]**2 - z_min[2]**2)

    return round(height*10,1)  


In [9]:
from  tqdm import tqdm
heights = []
for fname in tqdm(rgb_files,total=len(rgb_files)):
    rgb ,  depth , scale = get_image_data(fname)
    child_bbox = infer.detect_child(rgb)
    x1,y1,x2,y2 = child_bbox
    
    predicted_image = infer.inference_rgbimage(rgb_image=rgb[y1:y2,x1-20:x2+20],
                                               depth_image_size=depth.shape[:2],
                                               )
    
    pcd = create_point_cloud(predicted_image=predicted_image)
    pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
    pcd_o3d.estimate_normals()
    heights.append(get_approx_height(pcd_o3d))

100%|██████████| 26/26 [00:21<00:00,  1.19it/s]


In [10]:
print("Mean Height of the child {:.2f}".format(np.mean(heights)))

Mean Height of the child 117.64
